In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification, load_iris, load_wine
from trees.Stree import Stree
import time

In [2]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [3]:
import time
from sklearn.model_selection import train_test_split
from trees.Stree import Stree

random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state, stratify=y)
    return Xtrain, Xtest, ytrain, ytest

# data = load_creditcard(-5000) # Take all true samples + 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
data = load_creditcard(-1000) # Take all the samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (1492, 28)  y.shape (1492,)
Fraud: 33.311% 497
Valid: 66.689% 995


In [4]:
t = time.time()
for C in (.001, .01, 1, 5, 17):
    clf = Stree(C=C, random_state=random_state)
    clf.fit(Xtrain, ytrain)
    print(f"************** C={C} ****************************")
    print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
    print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
    print(clf)
    print(f"**************************************************")
print(f"{time.time() - t:.4f} secs")

************** C=0.001 ****************************
Classifier's accuracy (train): 0.9626
Classifier's accuracy (test) : 0.9487
root
root - Down
root - Down - Down, <cgaf> - Leaf class=1 belief=0.978261 counts=(array([0, 1]), array([  7, 315]))
root - Down - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([2]))
root - Up
root - Up - Down
root - Up - Down - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([1]))
root - Up - Down - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([1]))
root - Up - Up, <cgaf> - Leaf class=0 belief=0.955432 counts=(array([0, 1]), array([686,  32]))

**************************************************
************** C=0.01 ****************************
Classifier's accuracy (train): 0.9636
Classifier's accuracy (test) : 0.9509
root
root - Down
root - Down - Down, <cgaf> - Leaf class=1 belief=0.987421 counts=(array([0, 1]), array([  4, 314]))
root - Down - Up, <pure> - Leaf class=0 belief=1.000000 coun

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
scaler = StandardScaler()
cclf = CalibratedClassifierCV(base_estimator=LinearSVC(), cv=5)
cclf.fit(Xtrain, ytrain)
res = cclf.predict_proba(Xtest)
#an array containing probabilities of belonging to the 1st class

In [6]:
print(clf)

root
root - Down
root - Down - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([316]))
root - Down - Up
root - Down - Up - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([1]))
root - Down - Up - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([9]))
root - Up
root - Up - Down
root - Up - Down - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([2]))
root - Up - Down - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([4]))
root - Up - Up
root - Up - Up - Down
root - Up - Up - Down - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([4]))
root - Up - Up - Down - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([3]))
root - Up - Up - Up, <cgaf> - Leaf class=0 belief=0.964539 counts=(array([0, 1]), array([680,  25]))



In [7]:
from trees.Siterator import Siterator
it = Siterator(clf._tree)

In [8]:
while(it.hasNext()):
    print(it.next())

root

root - Down

root - Down - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([316]))

root - Up

root - Up - Down

root - Up - Down - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([2]))

root - Down - Up

root - Down - Up - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([1]))

root - Up - Up

root - Up - Up - Down

root - Up - Up - Down - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([4]))

root - Up - Down - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([4]))

root - Down - Up - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([9]))

root - Up - Up - Up, <cgaf> - Leaf class=0 belief=0.964539 counts=(array([0, 1]), array([680,  25]))

root - Up - Up - Down - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([3]))



In [1]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

scale = 8
# Make data.
X = np.arange(-scale, scale, 0.25)
Y = np.arange(-scale, scale, 0.25)
X, Y = np.meshgrid(X, Y)
Z = X**2 + Y**2

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                   linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(0, 100)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# rotate the axes and update
for angle in range(0, 360):
   ax.view_init(30, 40)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

ImportError: Failed to import any qt binding